In [ ]:
import sys
sys.path.append("../tests")
sys.path.append("../Szakdolgozat/tests")
import plot_from_json

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from arc_ahrm import ArcAHRM
from load_dataset import LoadDataset
import os
import random
import cv2
import numpy as np
import matplotlib.pyplot as plt
import json
from IPython.display import clear_output
from visualization import Visualization

from train import train, start

plt.style.use('dark_background')

script_directory = os.getcwd()

plt.style.use('dark_background')

def plt_show(img):
    clear_output(wait=False)
    plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    plt.show()

if __name__ == "__main__":
    start(plt_show)